In [2]:
from __future__ import division
from __future__ import print_function
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
import pandas as pd
import numpy  as np

In [3]:
data_dir = "./small-10-datasets/"
data_titles = ['accidents', 'baudio', 'bnetflix', 'dna', 'jester', 'kdd', 'msnbc',
              'nltcs', 'plants', 'r52']
test  = dict()
train = dict()
valid = dict()

for title in data_titles:
    test[title]  = np.loadtxt(data_dir + title + '.test.data', delimiter=',')
    train[title] = np.loadtxt(data_dir + title + '.ts.data', delimiter=',')
    valid[title] = np.loadtxt(data_dir + title + '.valid.data', delimiter=',')

In [4]:
# Bayesian Network, No edges Algorithm
# Takes in a dataset of binary variables and takes a test set of binary variables.
def BN_NE(T, test):
    cols = np.transpose(T)
    n    = len(cols)
    p_1 = np.array([(sum(cols[i] == 1) + 1) / (len(cols[i]) + n) for i in range(n)])
    return np.array([[p_1[i] if ti == 0 else 1 - p_1[i] for i, ti in enumerate(t)] for t in test])

class BN:
    #def __init__():
        
    #def fit():
        
    # Find Weights
    def FW(T):
        n = len(T)
        A = np.transpose(T)
        matrix = [[0 for i in range(n)] for j in range(n)]
        
    # Chow-Liu Algorithm
    #def CL():
    

# Log Likelihood
def LL(p):
    return np.sum(np.log(p))

In [132]:
ll = [LL(x) for x in BN_NE(train['accidents'],test['accidents'])]

In [134]:
sum(ll) / len(ll)

-202.1508858538403

In [123]:
#O(n^3)
def CL(T):
    d = np.transpose(T)
    n = len(d)
    p_0 = [(sum(d[i] == 0) + 1) / (len(d[i] + n)) for i in range(n)]
    p_1 = [(sum(d[i] == 1) + 1) / (len(d[i] + n)) for i in range(n)]
    def getAssociation(i, j):
        Ixy = 0
        if (i != j):
            for y in np.unique(d[j]):
                for x in np.unique(d[i]):
                    # P(x,y) = P(x | y) * p(y)
                    XGY = d[i][d[j] == y]
                    X   = d[i] == x
                    Y   = d[j] == y
                    py = (sum(Y) + 1) / (len(Y) + n)
                    px = (sum(X) + 1) / (len(X) + n)
                    pxGy = (sum(XGY) + 1) / (len(XGY) + n)
                    pxy = pxGy * py
                    Ixy += pxy * np.log(pxy / (px * py))
        return Ixy
    matrix = np.array([[getAssociation(i,j) for i in range(n)] for j in range(n)]).astype(float)
    MST = minimum_spanning_tree(matrix * np.array(-1)).toarray()
    return np.array([[0 if x == 0 else x * -1 for x in X] for X in MST])

def BN(T):
    CLT = CL(T)

In [124]:
M = CL(train['accidents'][0:10])